In [1]:
import json
import pandas as pd
import numpy as np
import qrisklab
from qrisklab.core.vol_models.wing_model import WingModel
from qrisklab.core.vol_models.svi_model import SVIModel, SVIPlot


pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


# Get and Clean Market Data (Deribit)

In [2]:
valuation_date = '2025-06-03' # this is used to calculate time_to_expiry

In [3]:
print("Fetching real-time Deribit option data...")
# dbc = qrisklab.clients.deribit.DeribitClient()
# df_real_time = dbc.fetch_deribit_option_data(currency="BTC")
df_real_time = pd.read_csv('deribit_option_data_real_time.csv')

if not df_real_time.empty:
    df_real_time.to_csv("deribit_option_data_real_time.csv", index=False)
    print("Real-time data saved to 'deribit_option_data_real_time.csv'")

Fetching real-time Deribit option data...
Real-time data saved to 'deribit_option_data_real_time.csv'


In [4]:
df_real_time['instrument'] = df_real_time['instrument_name']
df_processed = qrisklab.utils.process_instruments(df_real_time,valuation_date)
df_processed['vol'] = df_processed['mark_iv']/100
df_processed = df_processed[df_processed['put_call']=='call']
df_processed[['expiry','time_to_expiry','strike','vol','forward_price']]['expiry'].unique()


array(['2025-07-25', '2025-06-05', '2025-06-06', '2025-12-26',
       '2025-06-20', '2025-08-29', '2025-06-04', '2026-03-27',
       '2025-06-27', '2025-09-26', '2025-06-13'], dtype=object)

# Wing Model (Real Market Data)

In [ ]:
df = df_processed[df_processed['expiry']=='2025-06-20']

wm_ui = WingModel()
wm_ui.set_forward(df.forward_price.values[0])
wm_ui.set_market_data(df['strike'].values, df['vol'].values)
wm_ui.create_interactive_wing_fit_ui()

Button(button_style='success', description='Fit to Market Data', style=ButtonStyle())

Output()

# Wing Model (Synthetic Data)

In [6]:

# Create synthetic implied vol market data
strikes = np.linspace(60, 1400, 17)  # More strikes
true_params = dict(vr=0.20, sr=-0.1, pc=1.5, cc=0.8, dc=-0.3, uc=0.3, dsm=0.4, usm=0.4)
F = 1000
Ref = 1000
ATM = 1000

# Generate vols from the wing model with some noise
_, vols = WingModel.wing_vol_curve_with_smoothing(
    strikes=strikes,
    F=F,
    vr=true_params['vr'],
    sr=true_params['sr'],
    pc=true_params['pc'],
    cc=true_params['cc'],
    dc=true_params['dc'],
    uc=true_params['uc'],
    dsm=true_params['dsm'],
    usm=true_params['usm'],
    VCR=0,
    SCR=0,
    SSR=100,
    Ref=Ref,
    ATM=ATM
)
noisy_vols = vols + np.random.normal(0, 0.005, size=len(vols))  # Add small noise

# Create DataFrame and load into tool
df = pd.DataFrame({'strike': strikes, 'vol': noisy_vols})

wm_ui = WingModel()
wm_ui.set_forward(F)
wm_ui.set_market_data(df['strike'].values, df['vol'].values)
wm_ui.create_interactive_wing_fit_ui()

Button(button_style='success', description='Fit to Market Data', style=ButtonStyle())

Output()

# SVI (Real Market Data)

In [13]:

df = df_processed.copy()
df['IV'] = df['mark_iv'].astype(float)
df['Strike'] = df['strike'].astype(float)
df['Date'] = pd.to_datetime(df['expiry'])
df['Tau'] = df['time_to_expiry']
df['F'] = df['forward_price']
df['S'] = df['underlying_price']

df = df[df['Tau']>0]

svim = SVIModel(df[['IV','Strike','Date','Tau','F']])
df_fit = svim.fit(no_butterfly=False, no_calendar=False).reset_index(drop=False).rename(columns={'index':'Date'})
df_fit = pd.merge(df_fit, df[['Date','F','time_to_expiry']].drop_duplicates(subset=['Date'],keep='first'),on='Date',how='left')

plot_vol = SVIPlot()
plot_vol.allsmiles(svim)

{'a': np.float64(0.0001489932377888319), 'b': np.float64(0.007725691045195969), 'rho': np.float64(-0.18623984079611702), 'm': np.float64(-0.00010644468918448173), 'sigma': np.float64(0.02042487715004584)}
{'a': np.float64(-0.00027086384668644197), 'b': np.float64(0.017949319942554345), 'rho': np.float64(-0.2639456603413556), 'm': np.float64(-0.011934802687513048), 'sigma': np.float64(0.05057229246033972)}
{'a': np.float64(-0.0002894012000905987), 'b': np.float64(0.024839207306125707), 'rho': np.float64(-0.36459182084200004), 'm': np.float64(-0.028912511081638472), 'sigma': np.float64(0.05614021810552852)}
{'a': np.float64(-0.01277918389760015), 'b': np.float64(0.06407185448822705), 'rho': np.float64(0.005595081522019564), 'm': np.float64(0.02122641553445983), 'sigma': np.float64(0.2597657626533335)}
{'a': np.float64(-3.9292221335603115), 'b': np.float64(0.9999999634449134), 'rho': np.float64(0.38000546887138925), 'm': np.float64(1.7745897469712801), 'sigma': np.float64(4.25590297795438